In [1]:
import os.path as osp
import shutil
import os
import nimare
from nimare.extract import fetch_neurosynth
from nimare.io import convert_neurosynth_to_dataset
from glob import glob
import numpy as np
import pickle

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
/data/SFIMJGC_HCP7T/Apps/envs/fc_introspection/lib/python3.7/site-packages/nimare/__init__.py:74: FutureWarning: Python 3.6 and 3.7 support is deprecated and will be removed in release 0.1.0 of NiMARE. Consider switching to Python 3.8, 3.9 or 3.10.
  _python_deprecation_warnings()


In [2]:
print(nimare.__version__)

0.0.13


# 1. Folder Setup

In [3]:
PRJDIR = "/data/SFIMJGC_Introspec/2023_fc_introspection"
vocab = 'LDA400'

In [4]:
RESOURCE_NIMARE_DIR  = osp.join(PRJDIR,'nimare')
VOCAB_DIR            = osp.join(RESOURCE_NIMARE_DIR,vocab)
METAMAPS_ORIG_DIR    = osp.join(VOCAB_DIR,"meta-analyses-orig")  # where to save meta-analysis maps
METAMAPS_RPI_DIR     = osp.join(VOCAB_DIR,"meta-analyses-RPI")  # where to save meta-analysis maps

ns_dset_path         = os.path.join(VOCAB_DIR, f"neurosynth_dataset_{vocab}.pkl.gz")
print(ns_dset_path)

/data/SFIMJGC_Introspec/2023_fc_introspection/nimare/LDA400/neurosynth_dataset_LDA400.pkl.gz


In [5]:
# Create Empty Output Folders
# ===========================
print("++ INFO: Setting up all necessary folders")
for folder_path in [VOCAB_DIR, METAMAPS_ORIG_DIR]:
    if osp.exists(folder_path):
        print(" + WARNING: Removing folder [%s]" % folder_path)
        shutil.rmtree(folder_path)
    print(" + INFO: Generating/Regenerating output folder [%s]" % folder_path)
    os.mkdir(folder_path)

++ INFO: Setting up all necessary folders
 + WARNING: Removing folder [/data/SFIMJGC_Introspec/2023_fc_introspection/nimare/LDA400]
 + INFO: Generating/Regenerating output folder [/data/SFIMJGC_Introspec/2023_fc_introspection/nimare/LDA400]
 + INFO: Generating/Regenerating output folder [/data/SFIMJGC_Introspec/2023_fc_introspection/nimare/LDA400/meta-analyses-orig]


# 2. Download Neurosynth 7 database

In [6]:
# Download NeuroSynth database
print("++ INFO: Fetching neurosynth dataset for this vocabulary...")
files = fetch_neurosynth(data_dir=VOCAB_DIR, version="7", overwrite=False, vocab=vocab, source="abstract")

INFO:nimare.extract.utils:Dataset created in /data/SFIMJGC_Introspec/2023_fc_introspection/nimare/LDA400/neurosynth
INFO:nimare.extract.extract:Searching for any feature files matching the following criteria: [('vocab-LDA400', 'source-abstract', 'data-neurosynth', 'version-7')]


++ INFO: Fetching neurosynth dataset for this vocabulary...


# 3. Convert Neurosynth Database to NiMare Dataset

In [7]:
%%time
# Convert to NiMare Dataset
neurosynth_db = files[0]
neurosynth_dset = convert_neurosynth_to_dataset(
        coordinates_file=neurosynth_db['coordinates'],
        metadata_file=neurosynth_db['metadata'],
        annotations_files=neurosynth_db['features'],
        )

CPU times: user 2min 46s, sys: 1.73 s, total: 2min 48s
Wall time: 2min 42s


In [8]:
# Save the dataset as a pickle file to the Resources directory
print (" + Saving dataset to %s" % ns_dset_path)
neurosynth_dset.save(ns_dset_path)

 + Saving dataset to /data/SFIMJGC_Introspec/2023_fc_introspection/nimare/LDA400/neurosynth_dataset_LDA400.pkl.gz


In [9]:
# Extract Topic Names
topics_ORIG = neurosynth_dset.get_labels()
print(topics_ORIG[0:10])
print(len(topics_ORIG))

['LDA400_abstract_weight__0_exercise_igd_internet', 'LDA400_abstract_weight__1_context_contextual_contexts', 'LDA400_abstract_weight__2_sulcus_intraparietal_ips', 'LDA400_abstract_weight__3_individuals_resonance_magnetic', 'LDA400_abstract_weight__4_mood_affective_induction', 'LDA400_abstract_weight__5_judgments_judgment_judged', 'LDA400_abstract_weight__6_long_term_short', 'LDA400_abstract_weight__7_passive_viewing_listening', 'LDA400_abstract_weight__8_arousal_subjective_ratings', 'LDA400_abstract_weight__9_amplitude_spontaneous_frequency']
400


***

# 4. Train LDA Model

> Call S20s for 50 and 400 topics. This takes days to run and about 100GB of memory. Can be run on spersist node with 24 cpus using the NUMEXPR_MAX_THREADS environment variable.

In [10]:
lda_model_path = f'/data/SFIMJGC_Introspec/prj2021_dyneusr/Resources_NiMare/{vocab}/lda_model.pkl.gz'
with open(lda_model_path,'rb') as f:
    model_results = pickle.load(f)

In [11]:
neurosynth_EXTRA_dset = model_results['new_dset']

In [12]:
topics       = neurosynth_dset.get_labels()
topics_EXTRA = neurosynth_EXTRA_dset.get_labels()
print(len(topics), len(topics_EXTRA))

400 800


In [13]:
# Save the dataset as a pickle file to the Resources directory
ns_dset_EXTRA_path         = os.path.join(VOCAB_DIR, f"neurosynth_dataset_{vocab}_EXTRA.pkl.gz")
print (" + Saving dataset to %s" % ns_dset_EXTRA_path)
neurosynth_EXTRA_dset.save(ns_dset_EXTRA_path)

 + Saving dataset to /data/SFIMJGC_Introspec/2023_fc_introspection/nimare/LDA400/neurosynth_dataset_LDA400_EXTRA.pkl.gz


# 5. Generate Topic Maps

In [14]:
import pandas as pd
import os.path as osp
import os
from datetime import datetime
import getpass
import subprocess

from utils.basics import get_sbj_scan_list

from utils.basics import PRJ_DIR, DATA_DIR, SCRIPTS_DIR #NOTEBOOKS_DIR, RESOURCES_DINFO_DIR, PREPROCESSING_NOTES_DIR, 
print('++ INFO: Project Dir:                  %s' % PRJ_DIR) 
#print('++ INFO: Notebooks Dir:                %s' % NOTEBOOKS_DIR) 
print('++ INFO: Bash Scripts Dir:             %s' % SCRIPTS_DIR)
#print('++ INFO: Resources (Dataset Info) Dir: %s' % RESOURCES_DINFO_DIR)
#print('++ INFO: Pre-processing Notes Dir:     %s' % PREPROCESSING_NOTES_DIR)
print('++ INFO: Data Dir:                     %s' % DATA_DIR)

++ INFO: Project Dir:                  /data/SFIMJGC_Introspec/2023_fc_introspection
++ INFO: Bash Scripts Dir:             /data/SFIMJGC_Introspec/2023_fc_introspection/code/fc_introspection/bash
++ INFO: Data Dir:                     /data/SFIMJGC_Introspec/pdn


In [15]:
username = getpass.getuser()
print('++ INFO: user working now --> %s' % username)

++ INFO: user working now --> javiergc


In [16]:
#user specific folders
#=====================
swarm_folder   = osp.join(PRJ_DIR,'SwarmFiles.{username}'.format(username=username))
logs_folder    = osp.join(PRJ_DIR,'Logs.{username}'.format(username=username))

swarm_path     = osp.join(swarm_folder,f'S19_NiMareTopics_{vocab}.SWARM.sh')
logdir_path    = osp.join(logs_folder, f'S19_NiMareTopics_{vocab}.logs')
print('++ INFO: swarm_path = %s' % swarm_path)
print('++ INFO: logs dir   = %s' % logdir_path)

++ INFO: swarm_path = /data/SFIMJGC_Introspec/2023_fc_introspection/SwarmFiles.javiergc/S19_NiMareTopics_LDA400.SWARM.sh
++ INFO: logs dir   = /data/SFIMJGC_Introspec/2023_fc_introspection/Logs.javiergc/S19_NiMareTopics_LDA400.logs


In [17]:
# create user specific folders if needed
# ======================================
if not osp.exists(swarm_folder):
    os.makedirs(swarm_folder)
    print('++ INFO: New folder for swarm files created [%s]' % swarm_folder)
if not osp.exists(logdir_path):
    os.makedirs(logdir_path)
    print('++ INFO: New folder for log files created [%s]' % logdir_path)

In [18]:
# Open the file
swarm_file = open(swarm_path, "w")
# Log the date and time when the SWARM file is created
swarm_file.write('#Create Time: %s' % datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
swarm_file.write('\n')
# Insert comment line with SWARM command
swarm_file.write('#swarm -f {swarm_path} -g 32 -t 8 --partition quick,norm -b 5 --time 00:48:00 --logdir {logdir_path}'.format(swarm_path=swarm_path,logdir_path=logdir_path))
swarm_file.write('\n')

# Insert one line per subject
for topic in topics_EXTRA:
    topic = topic.replace(' ','-')
    swarm_file.write(f"export VOCAB={vocab} TOPIC={topic}; sh {SCRIPTS_DIR}/S19_NiMare_Create_TopicMaps.sh")
    swarm_file.write('\n')
swarm_file.close()

# 5. Flip them to RPI

> Need code to generate the master file

In [19]:
# Path to script that will generate the corrected version of the meta-analytic maps
nimare_flip_metamaps_script_path = osp.join(SCRIPTS_DIR,'STEP19_NiMare_Flip_TopicMaps.{vocab}.sh'.format(vocab=vocab))

# Path to fMRI Dataset to be used as reference when resampling. We pick the final data in MNI space from one random subject. All other subjects should be in the same space/grid
master_path = '/data/SFIMJGC_Introspec/2023_fc_introspection/nimare/NiMare_Decoding_Mask_GMribbon_2023.nii.gz'

In [20]:
nimare_metamaps_orig        = sorted(glob(os.path.join(METAMAPS_ORIG_DIR, f"{vocab}_*z_desc-specificity.nii.gz")))
print(len(nimare_metamaps_orig))

800


In [21]:
nimare_flip_metamaps_script = open(nimare_flip_metamaps_script_path, "w")
nimare_flip_metamaps_script.write('# Script to create flipped version of NiMare outputs\n')
nimare_flip_metamaps_script.write('module load afni\n')
nimare_flip_metamaps_script.write('\n')
nimare_flip_metamaps_script.write('set -e\n')
nimare_flip_metamaps_script.write('\n')
nimare_flip_metamaps_script.write('# Create output folder (if needed)\n')
nimare_flip_metamaps_script.write('if [ ! -d {out_folder} ]; then mkdir {out_folder}; fi\n\n'.format(out_folder=METAMAPS_RPI_DIR))
for orig_path in nimare_metamaps_orig:
    orig_file = osp.basename(orig_path)
    #orig_file = orig_file.replace(' ','\ ')
    new_path  = osp.join(METAMAPS_RPI_DIR,orig_file)
    #new_path  = new_path.replace(' ','-')
    nimare_flip_metamaps_script.write('3dLRflip -LR -overwrite -prefix "{out_file}" "{in_file}"\n'.format(in_file=orig_path, out_file=new_path))
    nimare_flip_metamaps_script.write('3drefit -orient RPI -space MNI "{in_file}"\n'.format(in_file=new_path))
    nimare_flip_metamaps_script.write('3dresample -overwrite -master {master_path} -input "{in_file}" -prefix "{out_file}"\n\n'.format(master_path = master_path,
                                                                                                                               in_file = new_path,
                                                                                                                               out_file = new_path))
nimare_flip_metamaps_script.close()

In [22]:
nimare_flip_metamaps_script_path

'/data/SFIMJGC_Introspec/2023_fc_introspection/code/fc_introspection/bash/STEP19_NiMare_Flip_TopicMaps.LDA400.sh'